In [2]:
# Packages to install 

!pip install -q selenium 
!pip install -q BeautifulSoup4
!pip install -q ipdb

In [1]:
import requests as rq
import bs4 as bs
import json
import traceback
import re
import numpy as np
import json
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import glob, os
import ipdb
from bs4 import BeautifulSoup

In [28]:
request = rq.get('https://live.bible.is/bible/ENGWWH/MAT/1')

html = request.content

In [29]:
html[:1000]

b'<!DOCTYPE html><html><head><meta charSet="utf-8" class="next-head"/><meta name="description" content="The book of the genealogy of Jesus Christ, the son of David, the son of Abraham...." class="next-head"/><meta property="og:title" content="Matthew 1 | Bible.is" class="next-head"/><meta property="og:image" content="https://live.bible.is/static/icon-310x310.png" class="next-head"/><meta property="og:image:width" content="310" class="next-head"/><meta property="og:image:height" content="310" class="next-head"/><meta property="og:url" content="https://live.bible.is//bible/ENGWWH/MAT/1" class="next-head"/><meta property="og:description" content="The book of the genealogy of Jesus Christ, the son of David, the son of Abraham...." class="next-head"/><meta name="twitter:title" content="Matthew 1" class="next-head"/><meta name="twitter:description" content="The book of the genealogy of Jesus Christ, the son of David, the son of Abraham...." class="next-head"/><title class="next-head">Matthew

In [30]:
soup = BeautifulSoup(html, 'html.parser') 

In [36]:
soup.body.text

'WWH - World English Bible (Winfred Henson)Matthew 11\xa01\xa0The book of the genealogy of Jesus Christ, the son of David, the son of Abraham.\xa02\xa0Abraham became the father of Isaac. Isaac became the father of Jacob. Jacob became the father of Judah and his brothers.\xa03\xa0Judah became the father of Perez and Zerah by Tamar. Perez became the father of Hezron. Hezron became the father of Ram.\xa04\xa0Ram became the father of Amminadab. Amminadab became the father of Nahshon. Nahshon became the father of Salmon.\xa05\xa0Salmon became the father of Boaz by Rahab. Boaz became the father of Obed by Ruth. Obed became the father of Jesse.\xa06\xa0Jesse became the father of King David. David became the father of Solomon by her who had been the wife of Uriah.\xa07\xa0Solomon became the father of Rehoboam. Rehoboam became the father of Abijah. Abijah became the father of Asa.\xa08\xa0Asa became the father of Jehoshaphat. Jehoshaphat became the father of Joram. Joram became the father of Uz

In [43]:
s = soup.find_all("span", {"class": ""})
# s
for t in s:
    print(t.text)

The book of the genealogy of Jesus Christ, the son of David, the son of Abraham.
Abraham became the father of Isaac. Isaac became the father of Jacob. Jacob became the father of Judah and his brothers.
Judah became the father of Perez and Zerah by Tamar. Perez became the father of Hezron. Hezron became the father of Ram.
Ram became the father of Amminadab. Amminadab became the father of Nahshon. Nahshon became the father of Salmon.
Salmon became the father of Boaz by Rahab. Boaz became the father of Obed by Ruth. Obed became the father of Jesse.
Jesse became the father of King David. David became the father of Solomon by her who had been the wife of Uriah.
Solomon became the father of Rehoboam. Rehoboam became the father of Abijah. Abijah became the father of Asa.
Asa became the father of Jehoshaphat. Jehoshaphat became the father of Joram. Joram became the father of Uzziah.
Uzziah became the father of Jotham. Jotham became the father of Ahaz. Ahaz became the father of Hezekiah.
Hezeki

In [108]:
Bible_dict = {
#     'GEN' : 50,
#     'EXO' : 40,
#     'LEV' : 27,
#     'NUM' : 36,
#     'DEU' : 34,
#     'JOS' : 24,
#     'JDG' : 21,
#     'RUT' : 4, 
#     '1SA' : 31,
#     '2SA' : 24,
#     '1KI' : 22,
#     '2KI' : 25,
#     '1CH' : 29,
#     '2CH' : 36,
#     'EZR' : 10,
#     'NEH' : 13,
#     'EST' : 10, 
#     'JOB' : 42,
#     'PSA' : 150,
#     'PRO' : 31,
#     'ECC' : 12,
#     'SNG' : 8,
#     'ISA' : 66,
#     'JER' : 52,
#     'LAM' : 5,
#     'EZK' : 48,
#     'DAN' : 12,
#     'HOS' : 14,
#     'JOL' : 3,
#     'AMO' : 9,
#     'OBA' : 1,
#     'JON' : 4,
#     'MIC' : 7,
#     'NAM' : 3,
#     'HAB' : 3,
#     'ZEP' : 3,
#     'HAG' : 2,
#     'ZEC' : 14,
#     'MAL' : 4,
    'MAT' : 28,
    'MRK' : 16,
    'LUK' : 24,
    'JHN' : 21,
    'ACT' : 28,
    'ROM' : 16,
    '1CO' : 16,
    '2CO' : 13,
    'GAL' : 6,
    'EPH' : 6,
    'PHP' : 4,
    'COL' : 4,
    '1TH' : 5,
    '2TH' : 3,
    '1TI' : 6,
    '2TI' : 4,
    'TIT' : 3,
    'PHM' : 1,
    'HEB' : 13,
    'JAS' : 5,
    '1PE' : 5,
    '2PE' : 3,
    '1JN' : 5,
    '2JN' : 1,
    '3JN' : 1,
    'JUD' : 1,
    'REV' : 22
}

In [56]:
def load_book_chapter(book = 'GEN', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        with open("./data/English_World_English_Bible_NT_Drama/B"+book+"_"+verse, "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

## Matthew

In [72]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B01___"+var+"_Matthew_____EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [73]:
Book_list = list(Bible_dict.keys())
Book_list[0]

'MAT'

In [74]:
Bible_dict['MAT']

28

In [75]:
book = Book_list[0]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## Mark

In [ ]:
B02___01_Mark________EN1WEBN2DA

In [76]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B02___"+var+"_Mark________EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [79]:
Book_list = list(Bible_dict.keys())
Book_list[1]

'MRK'

In [80]:
book = Book_list[1]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## Luke .. Acts

In [ ]:
B03___01_Luke________EN1WEBN2DA

In [94]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B05___"+var+"_Acts________EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [95]:
Book_list = list(Bible_dict.keys())
Book_list[4]

'ACT'

In [96]:
book = Book_list[4]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

Traceback (most recent call last):
  File "<ipython-input-94-9ebe078b1548>", line 22, in load_book_chapter_format
    if t.text.split()[0] == 'Copyrighted':
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-94-9ebe078b1548>", line 22, in load_book_chapter_format
    if t.text.split()[0] == 'Copyrighted':
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-94-9ebe078b1548>", line 22, in load_book_chapter_format
    if t.text.split()[0] == 'Copyrighted':
IndexError: list index out of range


## Romains

In [ ]:
B06___01_Romans______EN1WEBN2DA

In [99]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B06___"+var+"_Romans______EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [100]:
Book_list = list(Bible_dict.keys())
Book_list[5]

'ROM'

In [101]:
book = Book_list[5]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## 1Corinthians, 2Corinthians

In [ ]:
B07___01_1CorinthiansEN1WEBN2DA

In [109]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B08___"+var+"_2CorinthiansEN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [110]:
Book_list = list(Bible_dict.keys())
Book_list[7]

'2CO'

In [111]:
book = Book_list[7]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## Galatians ... Ephesians

In [ ]:
B09___01_Galatians___EN1WEBN2DA

In [115]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B10___"+var+"_Ephesians___EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [116]:
Book_list = list(Bible_dict.keys())
Book_list[9]

'EPH'

In [117]:
book = Book_list[9]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## Philippians

In [119]:
# B11___01_Philippians_EN1WEBN2DA
B12___01_Colossians__EN1WEBN2DA

In [123]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B12___"+var+"_Colossians_EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [124]:
Book_list = list(Bible_dict.keys())
Book_list[11]

'COL'

In [125]:
book = Book_list[11]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## Colossians

In [119]:
B12___01_Colossians__EN1WEBN2DA

In [126]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B12___"+var+"_Colossians__EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [127]:
Book_list = list(Bible_dict.keys())
Book_list[11]

'COL'

In [128]:
book = Book_list[11]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## 1Thess ... 2Thess

In [119]:
B13___01_1Thess______EN1WEBN2DA

In [132]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B14___"+var+"_2Thess______EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [133]:
Book_list = list(Bible_dict.keys())
Book_list[13]

'2TH'

In [134]:
book = Book_list[13]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## 1Timothy, 2Timothy

In [119]:
B15___01_1Timothy____EN1WEBN2DA

In [141]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B16___"+var+"_2Timothy____EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [142]:
Book_list = list(Bible_dict.keys())
Book_list[15]

'2TI'

In [143]:
book = Book_list[15]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## Titus

In [119]:
B17___01_Titus_______EN1WEBN2DA

In [144]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B17___"+var+"_Titus_______EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [145]:
Book_list = list(Bible_dict.keys())
Book_list[16]

'TIT'

In [146]:
book = Book_list[16]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## Philemon, 

In [119]:
# B18___01_Philemon____EN1WEBN2DA


In [147]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B18___"+var+"_Philemon____EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [148]:
Book_list = list(Bible_dict.keys())
Book_list[17]

'PHM'

In [149]:
book = Book_list[17]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## Hebrews

In [150]:
# B19___01_Hebrews_____EN1WEBN2DA

In [151]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B19___"+var+"_Hebrews_____EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [152]:
Book_list = list(Bible_dict.keys())
Book_list[18]

'HEB'

In [153]:
book = Book_list[18]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## James

In [154]:
# B20___01_James_______EN1WEBN2DA

In [155]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B20___"+var+"_James_______EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [156]:
Book_list = list(Bible_dict.keys())
Book_list[19]

'JAS'

In [157]:
book = Book_list[19]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## 1Peter, 2Peter

In [154]:
# B21___01_1Peter______EN1WEBN2DA

In [163]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B22___"+var+"_2Peter______EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [164]:
Book_list = list(Bible_dict.keys())
Book_list[21]

'2PE'

In [165]:
book = Book_list[21]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## 1John, 2John, 3John

In [154]:
# B23___01_1John_______EN1WEBN2DA

In [172]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B25___"+var+"_3John_______EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [173]:
Book_list = list(Bible_dict.keys())
Book_list[24]

'3JN'

In [174]:
book = Book_list[24]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## Jude

In [176]:
# B26___01_Jude________EN1WEBN2DA

In [177]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B26___"+var+"_Jude________EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [182]:
Book_list = list(Bible_dict.keys())
Book_list[25]

'JUD'

In [179]:
book = Book_list[25]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

## Revelation

In [180]:
# B27___01_Revelation__EN1WEBN2DA

In [183]:
def load_book_chapter_format(book = 'MAT', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r_engl = rq.get('https://live.bible.is/bible/ENGWWH/'\
                     +str(book)+'/'+str(verse)+'/')
        html_engl = r_engl.content
        soup_engl = BeautifulSoup(html_engl, 'html.parser') 
        s_engl = soup_engl.find_all("span", {"class": ""})
        
        var = verse if len(verse) == 2 else '0'+verse
        with open("./data/English_World_English_Bible_NT_Drama/B27___"+var+"_Revelation__EN1WEBN2DA.txt", "w") as text_file:
                
            for i, t in enumerate(s_engl):
                
                # this removes special caracters from the text# this removes special caracters from the text
                # text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
                if t.text.split()[0] == 'Copyrighted':
                    break
                text = re.sub(r"[+]", ' ', t.text) 
                # .strip() to delect any leading and trailing whitespace
                text_file.write(text.strip()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [184]:
Book_list = list(Bible_dict.keys())
Book_list[26]

'REV'

In [185]:
book = Book_list[26]
    
verses = Bible_dict[book]
for i in range(1, verses+1):
    load_book_chapter_format(book = book, verse = str(i))        

# Download Data

In [59]:
Bible_dict.keys()

dict_keys(['GEN', 'EXO', 'LEV', 'NUM', 'DEU', 'JOS', 'JDG', 'RUT', '1SA', '2SA', '1KI', '2KI', '1CH', '2CH', 'EZR', 'NEH', 'EST', 'JOB', 'PSA', 'PRO', 'ECC', 'SNG', 'ISA', 'JER', 'LAM', 'EZK', 'DAN', 'HOS', 'JOL', 'AMO', 'OBA', 'JON', 'MIC', 'NAM', 'HAB', 'ZEP', 'HAG', 'ZEC', 'MAL', 'MAT', 'MRK', 'LUK', 'JHN', 'ACT', 'ROM', '1CO', '2CO', 'GAL', 'EPH', 'PHP', 'COL', '1TH', '2TH', '1TI', '2TI', 'TIT', 'PHM', 'HEB', 'JAS', '1PE', '2PE', '1JN', '2JN', '3JN', 'JUD', 'REV'])

In [36]:
for book in Bible_dict.keys():
    
    verses = Bible_dict[book][1]
    for i in range(1, verses+1):
        load_book_chapeter(book = book, verse = str(i))        

### loading validation

In [41]:
# load_book_chapeter(book = 'genesis', verse = '1')


> <ipython-input-39-875dc819d8d3>(26)load_book_chapeter()
     25         ipdb.set_trace()
---> 26         if len(s_engl) != len(s_ling) :
     27 #             ipdb.set_trace()



ipdb>  c


Traceback (most recent call last):
  File "<ipython-input-39-875dc819d8d3>", line 26, in load_book_chapeter
    if len(s_engl) != len(s_ling) :
Exception: ('Verse', 'empty s')


In [38]:
# for book in Bible_dict_valid.keys():
#     verses = Bible_dict_valid[book][1]
#     for i in range(1, verses+1):
#         load_book_chapeter(book = book, verse = str(i),  mode = "valid")        

In [5]:
# sum(Bible_dict.values())

1179

# Put the data in MaseseSpeech format

In [3]:
v = '1'
filen = ''

file = '00'+v
file

'001'

In [15]:
def load_book_chapeter(book = 'masese', verse = '1'):
    """
    This function will help to save in text form, depening to the book and verse argument 
    given in parameter
    
    The data is maintly taken from the website https://www.jw.org/
    """
    try :
        r = rq.get('https://www.jw.org/ln/Biblioteke/biblia/bi12/mikanda/'\
                     +str(book)+'/'+str(verse)+'/')

        html = r.content
        soup = BeautifulSoup(html, 'html.parser') 
        s = soup.find_all('span', {'class' : 'style-b'})

        if s == []:
            # This may happen when the verse doesn't exist
            raise Exception('Verse', 'empty s')
        
        file_name = ''
        
#         verse = int(verse)+1
#         verse = str(verse)
        if len(verse) == 1:
            file_name = '00'+verse
        elif len(verse) == 2:
            file_name = '0'+verse
            
            
        #Proverb is the 20th book in the bible    
        with open("./020/020-"+file_name+".trans.txt", "w") as text_file:              

            for i, t in enumerate(s):
                
                # this removes special caracters from the text# this removes special caracters from the text
#                 text = re.sub(r"[^a-zA-Z]+", ' ', t.text) 
#                 ipdb.set_trace()
                text = re.sub(r"[+]", ' ', t.text) 
                
                number = str(i)
                if len(number) == 1:
                    number = '00'+number
                elif len(number) == 2:
                    number = '0'+number
                # .strip() to delect any leading and trailing whitespace
                text_file.write("020-"+file_name+"-"+number+" "+text[2:].strip().upper()+"\n") # to use the ligaikuma app, we need to format of in the .txt
        
    except Exception as inst:      
        traceback.print_exc()

In [17]:
book = 'masese'

for i in range(1, 31+1):
    load_book_chapeter(book = book, verse = str(i))        

In [23]:
!ls data/200624-102822_lin_053_elicit/

200624-102822_lin_053_elicit_0-metadata.json
200624-102822_lin_053_elicit_0.wav
200624-102822_lin_053_elicit_10-metadata.json
200624-102822_lin_053_elicit_10.wav
200624-102822_lin_053_elicit_11-metadata.json
200624-102822_lin_053_elicit_11.wav
200624-102822_lin_053_elicit_12-metadata.json
200624-102822_lin_053_elicit_12.wav
200624-102822_lin_053_elicit_13-metadata.json
200624-102822_lin_053_elicit_13.wav
200624-102822_lin_053_elicit_14-metadata.json
200624-102822_lin_053_elicit_14.wav
200624-102822_lin_053_elicit_15-metadata.json
200624-102822_lin_053_elicit_15.wav
200624-102822_lin_053_elicit_16-metadata.json
200624-102822_lin_053_elicit_16.wav
200624-102822_lin_053_elicit_17-metadata.json
200624-102822_lin_053_elicit_17.wav
200624-102822_lin_053_elicit_18-metadata.json
200624-102822_lin_053_elicit_18.wav
200624-102822_lin_053_elicit_19-metadata.json
200624-102822_lin_053_elicit_19.wav
200624-102822_lin_053_elicit_1-metadata.json
200624-102822_lin_053_elicit_1.wav
200624-102822_lin_05

In [48]:
path = "data/200624-102822_lin_053_elicit/"

wav = [file for file in sorted(os.listdir(path)) if file[-4:] == '.wav']
wav

['200624-102822_lin_053_elicit_0.wav',
 '200624-102822_lin_053_elicit_1.wav',
 '200624-102822_lin_053_elicit_10.wav',
 '200624-102822_lin_053_elicit_11.wav',
 '200624-102822_lin_053_elicit_12.wav',
 '200624-102822_lin_053_elicit_13.wav',
 '200624-102822_lin_053_elicit_14.wav',
 '200624-102822_lin_053_elicit_15.wav',
 '200624-102822_lin_053_elicit_16.wav',
 '200624-102822_lin_053_elicit_17.wav',
 '200624-102822_lin_053_elicit_18.wav',
 '200624-102822_lin_053_elicit_19.wav',
 '200624-102822_lin_053_elicit_2.wav',
 '200624-102822_lin_053_elicit_20.wav',
 '200624-102822_lin_053_elicit_21.wav',
 '200624-102822_lin_053_elicit_22.wav',
 '200624-102822_lin_053_elicit_23.wav',
 '200624-102822_lin_053_elicit_24.wav',
 '200624-102822_lin_053_elicit_25.wav',
 '200624-102822_lin_053_elicit_26.wav',
 '200624-102822_lin_053_elicit_27.wav',
 '200624-102822_lin_053_elicit_28.wav',
 '200624-102822_lin_053_elicit_29.wav',
 '200624-102822_lin_053_elicit_3.wav',
 '200624-102822_lin_053_elicit_30.wav',
 '20

In [49]:
sorted(wav, key=lambda x: int(x.split('_')[-1].split('.')[0]))

['200624-102822_lin_053_elicit_0.wav',
 '200624-102822_lin_053_elicit_1.wav',
 '200624-102822_lin_053_elicit_2.wav',
 '200624-102822_lin_053_elicit_3.wav',
 '200624-102822_lin_053_elicit_4.wav',
 '200624-102822_lin_053_elicit_5.wav',
 '200624-102822_lin_053_elicit_6.wav',
 '200624-102822_lin_053_elicit_7.wav',
 '200624-102822_lin_053_elicit_8.wav',
 '200624-102822_lin_053_elicit_9.wav',
 '200624-102822_lin_053_elicit_10.wav',
 '200624-102822_lin_053_elicit_11.wav',
 '200624-102822_lin_053_elicit_12.wav',
 '200624-102822_lin_053_elicit_13.wav',
 '200624-102822_lin_053_elicit_14.wav',
 '200624-102822_lin_053_elicit_15.wav',
 '200624-102822_lin_053_elicit_16.wav',
 '200624-102822_lin_053_elicit_17.wav',
 '200624-102822_lin_053_elicit_18.wav',
 '200624-102822_lin_053_elicit_19.wav',
 '200624-102822_lin_053_elicit_20.wav',
 '200624-102822_lin_053_elicit_21.wav',
 '200624-102822_lin_053_elicit_22.wav',
 '200624-102822_lin_053_elicit_23.wav',
 '200624-102822_lin_053_elicit_24.wav',
 '200624-1

In [32]:
sorted??

Signature: sorted(iterable, /, *, key=None, reverse=False)
Docstring:
Return a new list containing all items from the iterable in ascending order.

A custom key function can be supplied to customize the sort order, and the
reverse flag can be set to request the result in descending order.
Type:      builtin_function_or_method


In [52]:
!ls

 020    LICENSE		    Main_MaseseSpeech_ASR.ipynb
 data  'Lingala JW.ipynb'   README.md


In [84]:
# Function to rename multiple files
# path = "data/200624-102822_lin_053_elicit/" # 1

# path = "data/200624-104423_lin_053_elicit/" # 2

# path = "data/200625-001111_lin_053_elicit/" # 3
# path = "data/200625-005243_lin_053_elicit/" # 4
# path = "data/200629-223643_lin_053_elicit/" # 5
# path = "data/200629-231545_lin_053_elicit/" # 6
# path = "data/200629-234022_lin_053_elicit/" # 7
# path = "data/200630-001335_lin_053_elicit/" # 8
# path = "data/200630-005007_lin_053_elicit/" # 9
# path = "data/200630-011337_lin_053_elicit/" # 10
# path = "data/200630-013828_lin_053_elicit/" # 11
# path = "data/200702-103352_lin_053_elicit/" # 12
# path = "data/200702-105438_lin_053_elicit/" # 13
# path = "data/200702-110714_lin_053_elicit/" # 14
# path = "data/200702-112925_lin_053_elicit/" # 15
# path = "data/200702-141619_lin_053_elicit/" # 16
# path = "data/200702-144002_lin_053_elicit/" # 17
# path = "data/200702-145857_lin_053_elicit/" # 18
# path = "data/200702-151139_lin_053_elicit/" # 19
# path = "data/200702-152742_lin_053_elicit/" # 20
path = "data/200702-154607_lin_053_elicit/" # 21


# dest = "data/020/"

wav = [file for file in sorted(os.listdir(path)) if file[-4:] == '.wav']

for i, filename in enumerate(sorted(wav, key=lambda x: int(x.split('_')[-1].split('.')[0]))):

    number = str(i)
    if len(number) == 1:
        number = '00'+number
    elif len(number) == 2:
        number = '0'+number

    my_dest ="020-021-" + number + ".wav"
    my_source = path + filename
    my_dest = path + my_dest
    # rename() function will
    # rename all the files
#     ipdb.set_trace()
    os.rename(my_source, my_dest) # I may need in the future to change this by copy (TODO)
#         i += 1

In [57]:
os.rename?

Signature: os.rename(src, dst, *, src_dir_fd=None, dst_dir_fd=None)
Docstring:
Rename a file or directory.

If either src_dir_fd or dst_dir_fd is not None, it should be a file
  descriptor open to a directory, and the respective path string (src or dst)
  should be relative; the path will then be relative to that directory.
src_dir_fd and dst_dir_fd, may not be implemented on your platform.
  If they are unavailable, using them will raise a NotImplementedError.
Type:      builtin_function_or_method


In [59]:
# # Function to rename multiple files
# path = "test/"
# dest = "data/020/"

# wav = [file for file in sorted(os.listdir(path)) if file[-4:] == '.wav']

# for i, filename in enumerate(sorted(wav, key=lambda x: int(x.split('_')[-1].split('.')[0]))):

#     number = str(i)
#     if len(number) == 1:
#         number = '00'+number
#     elif len(number) == 2:
#         number = '0'+number

#     my_dest ="020-001-" + number + ".wav"
#     my_source =path + filename
#     my_dest =path + my_dest
#     # rename() function will
#     # rename all the files
# #     ipdb.set_trace()
#     os.rename(my_source, my_dest)
# #         i += 1

In [9]:
os.path.exists('./scripts/check-verses.py')

True